# S2RA_mask_ETBFcrop_regrid025

**Date:** <br>
19 February 2022 <br>
**Background:** <br>
Issue -  <br>
**Author(s):**<br>
Thomas Moore<br>

### OOD documentation
https://opus.nci.org.au/display/DAE/Setting+up+a+Dask+Cluster+on+OOD

In [1]:
Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

In [2]:
import xarray as xr
import numpy as np
import xrft
import xesmf as xe
import scipy
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import os
import re
import cartopy.crs as ccrs
import proplot as pplt
from rechunker import rechunk
%config Completer.use_jedi = False

## import helper

In [3]:
import importlib.util
spec = importlib.util.spec_from_file_location("helper", "/g/data/v14/tm4888/code/helper-py/helper_tools.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)

## OOD cluster

In [4]:
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=2,processes=1,memory="47GB",walltime='02:45:00')
client = Client(cluster)
cluster.scale(cores=24)

/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.146:40249,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Load Tgrid data from zarr

In [10]:
Tgrid = xr.open_zarr('/g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.ocean.nativeTgrid.zarr',consolidated=True)

# Masking
## BoM ACCESS-S2 mask file isn't available so use where SST == exactly zero

# Load U from zarr

In [11]:
Ugrid = xr.open_zarr('/g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.ocean.u.nativeUgrid.zarr',consolidated=True)

# Land Mask U
## BoM ACCESS-S2 mask file isn't available so use where values == exactly zero

In [38]:
u_mask = one_u.where(one_u == 0)

# setup regridder using `xesmf`

In [11]:
u = u.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
v = v.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})

## curvilinear grid

In [12]:
nav_lon = u['lon'].compute()
nav_lat = u['lat'].compute()

# CHOOSE YOUR NEW GRID // define uniform 1x1 grid

In [13]:
new_1deg_grid = xe.util.grid_global(1, 1)
new_1deg_grid

<xarray.Dataset>
Dimensions:  (y: 180, x: 360, y_b: 181, x_b: 361)
Coordinates:
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
    lon_b    (y_b, x_b) int64 -180 -179 -178 -177 -176 ... 176 177 178 179 180
    lat_b    (y_b, x_b) int64 -90 -90 -90 -90 -90 -90 -90 ... 90 90 90 90 90 90
Dimensions without coordinates: y, x, y_b, x_b
Data variables:
    *empty*

# U

In [14]:
u = u.chunk({'y':1021, 'x':1442}) #xesmf hates chunks across grid vars?

In [15]:
%%time
regridder_S2_1deg_Ugrid = xe.Regridder(u, new_1deg_grid,'bilinear', ignore_degenerate=True, periodic=True,reuse_weights=False)

CPU times: user 1min 6s, sys: 2.95 s, total: 1min 9s
Wall time: 1min 20s


## EARTH SYSTEM MODELLING FRAMEWORK (NCAR / NOAA) - https://en.wikipedia.org/wiki/Earth_System_Modeling_Framework </br></br>  xESMF // https://github.com/JiaweiZhuang/xESMF/blob/master/README.rst

## $\bigstar$ solution to parallel regridding issue = https://github.com/JiaweiZhuang/xESMF/issues/71

#### regrid the full dataset object

In [16]:
%%time
regridder_S2_1deg_Ugrid._grid_in = None
regridder_S2_1deg_Ugrid._grid_out = None
ds_ACCESS_S2_RA_ocean_u_1degRegrid = regridder_S2_1deg_Ugrid(u)

CPU times: user 4.35 s, sys: 209 ms, total: 4.56 s
Wall time: 5.78 s


In [17]:
ds_ACCESS_S2_RA_ocean_u_1degRegrid

,Array,Chunk
Bytes,8.69 GiB,118.65 MiB
Shape,"(480, 75, 180, 360)","(480, 1, 180, 360)"
Count,2403 Tasks,75 Chunks
Type,float32,numpy.ndarray


# V

In [18]:
v = v.chunk({'y':1021, 'x':1442}) #xesmf hates chunks across grid vars?

In [19]:
%%time
regridder_S2_1deg_Vgrid = xe.Regridder(v, new_1deg_grid,'bilinear', ignore_degenerate=True, periodic=True,reuse_weights=False)

CPU times: user 1min 6s, sys: 1.1 s, total: 1min 7s
Wall time: 1min 18s


## EARTH SYSTEM MODELLING FRAMEWORK (NCAR / NOAA) - https://en.wikipedia.org/wiki/Earth_System_Modeling_Framework </br></br>  xESMF // https://github.com/JiaweiZhuang/xESMF/blob/master/README.rst

## $\bigstar$ solution to parallel regridding issue = https://github.com/JiaweiZhuang/xESMF/issues/71

#### regrid the full dataset object

In [20]:
%%time
regridder_S2_1deg_Vgrid._grid_in = None
regridder_S2_1deg_Vgrid._grid_out = None
ds_ACCESS_S2_RA_ocean_v_1degRegrid = regridder_S2_1deg_Vgrid(v)

CPU times: user 257 ms, sys: 2.99 ms, total: 260 ms
Wall time: 245 ms


In [21]:
ds_ACCESS_S2_RA_ocean_v_1degRegrid

,Array,Chunk
Bytes,8.69 GiB,118.65 MiB
Shape,"(480, 75, 180, 360)","(480, 1, 180, 360)"
Count,2403 Tasks,75 Chunks
Type,float32,numpy.ndarray


In [22]:
ds_ACCESS_S2_RA_ocean_v_1degRegrid = ds_ACCESS_S2_RA_ocean_v_1degRegrid.assign_coords(
    {'x': ds_ACCESS_S2_RA_ocean_v_1degRegrid.lon.isel(y=0),
    'y': ds_ACCESS_S2_RA_ocean_v_1degRegrid.lat.isel(x=0)}).drop(
    ['lon','lat']).rename({'x':'lon','y':'lat'})

In [23]:
ds_ACCESS_S2_RA_ocean_u_1degRegrid = ds_ACCESS_S2_RA_ocean_u_1degRegrid.assign_coords(
    {'x': ds_ACCESS_S2_RA_ocean_u_1degRegrid.lon.isel(y=0),
    'y': ds_ACCESS_S2_RA_ocean_u_1degRegrid.lat.isel(x=0)}).drop(
    ['lon','lat']).rename({'x':'lon','y':'lat'})

# write out the 1deg u & v zarr collections

In [24]:
%%time
!rm -rf /g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.ocean.u.regrid_1deg.zarr
ds_ACCESS_S2_RA_ocean_u_1degRegrid.to_zarr('/g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.ocean.u.regrid_1deg.zarr',
                                           consolidated=True)

CPU times: user 39.1 s, sys: 3.04 s, total: 42.1 s
Wall time: 2min 31s


In [25]:
%%time
!rm -rf /g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.ocean.v.regrid_1deg.zarr
ds_ACCESS_S2_RA_ocean_v_1degRegrid.to_zarr('/g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.ocean.v.regrid_1deg.zarr',
                                           consolidated=True)

CPU times: user 35.1 s, sys: 2.79 s, total: 37.9 s
Wall time: 2min 19s


# Load 1deg U&V

In [26]:
ds_u = xr.open_zarr('/g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.ocean.u.regrid_1deg.zarr')
ds_v = xr.open_zarr('/g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.ocean.v.regrid_1deg.zarr')

In [27]:
def calc_EKE(u, v, time_name = 'time'):
    '''
    Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}
    u,v are x and y currents as an xarray data array
    '''
    u_mean = u.mean(time_name)
    v_mean = v.mean(time_name)
    MKE = 0.5*(u_mean**2 + v_mean**2).rename('MKE') # currents
    EKE = ( 0.5 * ((u-u_mean)**2 + (v-v_mean)**2) ).rename('EKE') # eddies
    return EKE, MKE

In [28]:
u_da = ds_u.u
v_da = ds_v.v

In [29]:
%%time
[EKE, MKE] = calc_EKE(u_da, v_da, time_name='time')

CPU times: user 41.6 ms, sys: 3.89 ms, total: 45.5 ms
Wall time: 44.6 ms


In [30]:
KE = xr.merge([EKE,MKE])

In [31]:
%%time
!rm -rf /g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.EKE.regrid_1deg.zarr
KE.to_zarr('/g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.EKE.regrid_1deg.zarr',consolidated=True)

CPU times: user 3.88 s, sys: 472 ms, total: 4.35 s
Wall time: 14.6 s


# $ The\ End$

# Break glass in case of emergency
# $\Downarrow$

In [ ]:
client.restart()

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7fcb04415430>>, <Task finished name='Task-304880' coro=<Scheduler.restart() done, defined at /g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/distributed/scheduler.py:5885> exception=CommClosedError("Exception while trying to call remote method 'restart' before comm was established.")>)
Traceback (most recent call last):
  File "/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/distributed/comm/tcp.py", line 205, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/distributed/core.py", line 819, in send_recv_from_rpc
    result = awa

In [94]:
client.shutdown()

KeyboardInterrupt: 

In [93]:
client.restart()

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7fe51c7dee20>>, <Task finished name='Task-346239' coro=<Scheduler.restart() done, defined at /g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/distributed/scheduler.py:5885> exception=CommClosedError("Exception while trying to call remote method 'restart' before comm was established.")>)
Traceback (most recent call last):
  File "/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/distributed/comm/tcp.py", line 205, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/distributed/core.py", line 819, in send_recv_from_rpc
    result = awa

<Client: 'tcp://10.0.128.149:42147' processes=11 threads=22, memory=481.47 GiB>